An AI-powered assistant that interacts with users to provide bookstore-related services, including retrieving book prices and listing available books. The assistant leverages OpenAI's chat completions API and integrates dynamic tool handling for seamless responses to user queries.

In [3]:
import os
import json
from openai import OpenAI

In [4]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:00


In [5]:
import gradio as gr
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'API')
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [66]:
system_message = "You are a helpful and kind assistant for an bookstore called BOOKNOW. "
system_message += "you should ask questions to collect information and help them as best as you can. "
system_message += "they will ask about a book if you have information about it, help them .Always be accurate. If you don't know the answer, say so."

In [16]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content


In [97]:
books = {
    "To Kill a Mockingbird": "$15.99",
    "1984": "$13.49",
    "Pride and Prejudice": "$10.99",
    "The Great Gatsby": "$14.99",
    "Moby-Dick": "$17.50",
    "The Catcher in the Rye": "$12.00",
    "The Alchemist": "$11.99",
    "Brave New World": "$13.75",
    "The Lord of the Rings": "$25.00",
    "Harry Potter and the Sorcerer’s Stone": "$19.99",
    "The Kite Runner": "$14.25",
    "Sapiens: A Brief History of Humankind": "$21.50",
    "Becoming": "$18.00",
    "The Power of Habit": "$16.49",
    "Atomic Habits": "$18.75",
    "Educated": "$15.49",
    "The Silent Patient": "$12.99",
    "The Subtle Art of Not Giving a F*ck": "$14.00",
    "Thinking, Fast and Slow": "$22.50",
    "The 7 Habits of Highly Effective People": "$19.49"
}

def get_price(book):
    print(f"Tool get_price called for {book}")
    return books.get(book, 'Unknown')
def avalible_books():
    return list(books.keys())

In [99]:
get_price("The 7 Habits of Highly Effective People")

Tool get_price called for The 7 Habits of Highly Effective People


'$19.49'

In [98]:
avalible_books()

['To Kill a Mockingbird',
 '1984',
 'Pride and Prejudice',
 'The Great Gatsby',
 'Moby-Dick',
 'The Catcher in the Rye',
 'The Alchemist',
 'Brave New World',
 'The Lord of the Rings',
 'Harry Potter and the Sorcerer’s Stone',
 'The Kite Runner',
 'Sapiens: A Brief History of Humankind',
 'Becoming',
 'The Power of Habit',
 'Atomic Habits',
 'Educated',
 'The Silent Patient',
 'The Subtle Art of Not Giving a F*ck',
 'Thinking, Fast and Slow',
 'The 7 Habits of Highly Effective People']

In [100]:
price_function = {
    "name": "get_price",
    "description": "Get the price of the book. Call this whenever you need to know the ticket price. for example when they say 'how much for the book'",
    "parameters": {
        "type": "object",
        "properties": {
            "book": {
                "type": "string",
                "description": "The book they want  price of",
            },
        },
        "required": ["book"],
        "additionalProperties": False
    }
}
avaliblality_function = {
    "name": "avalible_books",
    "description": "Get the list of avalible books,ll this whenever you need to know what books are avalible at the store. you can check if a book is avalible or not by having all the books avalible",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False
    }
}

In [101]:
tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": avaliblality_function}]

In [102]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, book = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [103]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    tool_name = tool_call.function.name  # Identify which tool was called

    if tool_name == "get_price":
        book = arguments.get('book')
        price = get_price(book)
        content = {"book": book, "price": price}
    elif tool_name == "avalible_books":
        content = {"available_books": avalible_books()}
    else:
        content = {"error": f"Tool {tool_name} is not supported."}

    response = {
        "role": "tool",
        "content": json.dumps(content),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, content

In [104]:
gr.ChatInterface(fn=chat, type="messages").launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://504a026cb6055a610b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
